In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [6]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv1 = nn.Sequential(                  #in_s, out_s, rf_in, rf_out
            nn.Conv2d(1, 8, 3, padding=1),           #28*28, 28*28, 1,3
            nn.ReLU(),
            nn.BatchNorm2d(8),
            nn.Dropout(0.1),
            nn.Conv2d(8, 8, 3, padding=1),           #28*28, 28*28, 3,5
            nn.ReLU(),
            nn.BatchNorm2d(8),
            nn.Conv2d(8, 20, 3, padding=1),           #28*28, 28*28, 5,7
            nn.ReLU(),
            nn.BatchNorm2d(20),
            # nn.Dropout(0.1),
            nn.MaxPool2d(2, 2)                         #28*28, 14*14, 7,8
            # nn.Dropout(0.1)
        )
        
        self.conv2 = nn.Sequential(
            nn.Conv2d(20, 16, 1),                       #14*14, 14*14, 8,8
            # nn.ReLU(), -->why relu. aggregators dont need relu to be appplied
            nn.BatchNorm2d(16),
            nn.Dropout(0.1),
            nn.Conv2d(16, 16, 3, padding=1),           #14*14, 14*14, 8,12
            nn.ReLU(),
            nn.BatchNorm2d(16),
            # nn.Dropout(0.1),
            
            nn.Conv2d(16, 30, 3, padding=1),           #14*14, 14*14, 12,16
            nn.ReLU(),
            nn.BatchNorm2d(30),

            nn.MaxPool2d(2, 2),                         #14*14, 7*7, 16,18
            # nn.Dropout(0.1)
            # nn.MaxPool2d(2, 2)
            # nn.Dropout(0.25)
        )
        
        self.conv3 = nn.Sequential(
            nn.Conv2d(30, 20, 1),                       #7*7, 7*7, 18,18
            # nn.ReLU(),
            nn.BatchNorm2d(20),
            nn.Conv2d(20, 20, 3, padding=1),           #7*7, 7*7, 18,26
            nn.ReLU(),
            nn.BatchNorm2d(20),
            nn.Conv2d(20, 32, 3, padding=1),           #7*7, 7*7, 26,34
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.Conv2d(32, 10, 1),                     #7*7, 7*7, 34,34
            # nn.ReLU(),
            nn.AvgPool2d(7)                            #7*7, 7*7, 34,34
            # nn.AdaptiveMaxPool2d(7)
            # nn.ReLU(),
            # nn.BatchNorm2d(128),
            # nn.MaxPool2d(2, 2)
            # nn.Dropout(0.25)
        )
        
        # self.fc = nn.Sequential(
        #     nn.Linear(16*7*7, 10)
        # )
                
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        # print(x.shape)
        x = x.view(-1, 10)
        
        # x = x.view(-1, 16*7*7)
        # x = self.fc(x)
        x = F.log_softmax(x, dim=1)
        return x

In [5]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 28, 28]              80
              ReLU-2            [-1, 8, 28, 28]               0
       BatchNorm2d-3            [-1, 8, 28, 28]              16
           Dropout-4            [-1, 8, 28, 28]               0
            Conv2d-5            [-1, 8, 28, 28]             584
              ReLU-6            [-1, 8, 28, 28]               0
       BatchNorm2d-7            [-1, 8, 28, 28]              16
            Conv2d-8           [-1, 20, 28, 28]           1,460
              ReLU-9           [-1, 20, 28, 28]               0
      BatchNorm2d-10           [-1, 20, 28, 28]              40
        MaxPool2d-11           [-1, 20, 14, 14]               0
           Conv2d-12           [-1, 16, 14, 14]             336
             ReLU-13          

In [5]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


100%|██████████| 9912422/9912422 [00:00<00:00, 460667596.42it/s]

Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw


100%|██████████| 28881/28881 [00:00<00:00, 124241737.26it/s]


Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 209008776.23it/s]


Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 20729628.69it/s]

Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw



In [6]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [6]:

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 8, 3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(8),
            nn.Dropout(0.1),
            nn.Conv2d(8, 8, 3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(8),
            nn.Conv2d(8, 20, 3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(20),
            # nn.Dropout(0.1),
            nn.MaxPool2d(2, 2)
            # nn.Dropout(0.1)
        )
        
        self.conv2 = nn.Sequential(
            nn.Conv2d(20, 20, 1),
            nn.ReLU(),
            nn.BatchNorm2d(20),
            nn.Dropout(0.1),
            nn.Conv2d(20, 20, 3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(20),
            # nn.Dropout(0.1),
            
            nn.Conv2d(20, 20, 3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(20),

            nn.MaxPool2d(2, 2),
            # nn.Dropout(0.1)
            # nn.MaxPool2d(2, 2)
            # nn.Dropout(0.25)
        )
        
        self.conv3 = nn.Sequential(
            nn.Conv2d(20, 32, 3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.Conv2d(32, 10, 1),
            nn.ReLU(),
            nn.AvgPool2d(7)
            # nn.AdaptiveMaxPool2d(7)
            # nn.ReLU(),
            # nn.BatchNorm2d(128),
            # nn.MaxPool2d(2, 2)
            # nn.Dropout(0.25)
        )
        
        # self.fc = nn.Sequential(
        #     nn.Linear(16*7*7, 10)
        # )
                
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        # print(x.shape)
        x = x.view(-1, 10)
        
        # x = x.view(-1, 16*7*7)
        # x = self.fc(x)
        x = F.log_softmax(x, dim=1)
        return x
        
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 20):
    # print(epoch)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

loss=0.08291856199502945 batch_id=468: 100%|██████████| 469/469 [00:25<00:00, 18.37it/s]



Test set: Average loss: 0.1516, Accuracy: 9577/10000 (96%)



loss=0.05580149218440056 batch_id=468: 100%|██████████| 469/469 [00:21<00:00, 21.60it/s]



Test set: Average loss: 0.0766, Accuracy: 9772/10000 (98%)



loss=0.08253753185272217 batch_id=468: 100%|██████████| 469/469 [00:22<00:00, 21.23it/s]


...


Test set: Average loss: 0.0251, Accuracy: 9917/10000 (99%)



loss=0.01232012640684843 batch_id=468: 100%|██████████| 469/469 [00:20<00:00, 22.77it/s]



Test set: Average loss: 0.0245, Accuracy: 9927/10000 (99%)



loss=0.02223578281700611 batch_id=468: 100%|██████████| 469/469 [00:21<00:00, 22.18it/s]



Test set: Average loss: 0.0254, Accuracy: 9915/10000 (99%)



In [7]:
'''
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 8, 3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(8),
            nn.Dropout(0.1),
            nn.Conv2d(8, 8, 3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(8),
            nn.Conv2d(8, 20, 3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(20),
            # nn.Dropout(0.1),
            nn.MaxPool2d(2, 2)
            # nn.Dropout(0.1)
        )
        
        self.conv2 = nn.Sequential(
            nn.Conv2d(20, 16, 1),
            # nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout(0.1),
            nn.Conv2d(16, 16, 3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            # nn.Dropout(0.1),
            
            nn.Conv2d(16, 30, 3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(30),

            nn.MaxPool2d(2, 2),
            # nn.Dropout(0.1)
            # nn.MaxPool2d(2, 2)
            # nn.Dropout(0.25)
        )
        
        self.conv3 = nn.Sequential(
            nn.Conv2d(30, 20, 1),
            # nn.ReLU(),
            nn.BatchNorm2d(20),
            nn.Conv2d(20, 20, 3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(20),
            nn.Conv2d(20, 32, 3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.Conv2d(32, 10, 1),
            # nn.ReLU(),
            nn.AvgPool2d(7)
            # nn.AdaptiveMaxPool2d(7)
            # nn.ReLU(),
            # nn.BatchNorm2d(128),
            # nn.MaxPool2d(2, 2)
            # nn.Dropout(0.25)
        )
        
        # self.fc = nn.Sequential(
        #     nn.Linear(16*7*7, 10)
        # )
                
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        # print(x.shape)
        x = x.view(-1, 10)
        
        # x = x.view(-1, 16*7*7)
        # x = self.fc(x)
        x = F.log_softmax(x, dim=1)
        return x
'''

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 20):
    # print(epoch)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

loss=0.1809646636247635 batch_id=468: 100%|██████████| 469/469 [00:25<00:00, 18.35it/s]



Test set: Average loss: 0.1077, Accuracy: 9714/10000 (97%)



loss=0.0505506657063961 batch_id=468: 100%|██████████| 469/469 [00:21<00:00, 21.94it/s]



Test set: Average loss: 0.0626, Accuracy: 9807/10000 (98%)



loss=0.030217520892620087 batch_id=468: 100%|██████████| 469/469 [00:21<00:00, 21.69it/s]


...


Test set: Average loss: 0.0199, Accuracy: 9941/10000 (99%)



loss=0.002575006103143096 batch_id=468: 100%|██████████| 469/469 [00:20<00:00, 22.55it/s]



Test set: Average loss: 0.0214, Accuracy: 9934/10000 (99%)



loss=0.026715941727161407 batch_id=468: 100%|██████████| 469/469 [00:20<00:00, 22.46it/s]



Test set: Average loss: 0.0202, Accuracy: 9942/10000 (99%)

